In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import os
from shapely.wkt import loads
from utils import clear_column_names



1 Bundesnetzagentur



In [2]:
# download excel file
df_1 = pd.read_excel('data/sources/Bundesnetzagentur/bundesnetzagentur_power_plants.xlsx', engine='openpyxl')

In [3]:
# Extract the file name
file_name = os.path.basename('data/sources/Bundesnetzagentur/bundesnetzagentur_power_plants.xlsx')
# Add a new column with the file name
df_1['Source'] = file_name

In [4]:
# change date format from DD.MM.YYYY to a datetime64 format YYYY-MM-DD
df_1['Datum der erstmaligen Inbetriebnahme der Einheit'] = pd.to_datetime(df_1['Datum der erstmaligen Inbetriebnahme der Einheit'], errors='coerce')

In [5]:
# for geocoding, rename German titles of the columns to English ones:
# 'Street', 'House Number', 'City', 'Postcode', 'State'
df_1 = df_1.rename(columns={
    'Straße der Einheit': 'Street',
    'Hausnummer der Einheit': 'House Number',
    'Ort der Einheit': 'City',
    'PLZ der Einheit': 'Postcode',
    'Bundesland der Einheit': 'State'
})

In [6]:
# filter only records for Bayern
df_1 = df_1[df_1['State'] == 'Bayern']

In [7]:
# delete rows with empty values in the columns which form an address for Nominatim API query
df_filtered = df_1.dropna(subset=['Street', 'House Number', 'City', 'Postcode'])

In [8]:
# concatenate column values in a string containing full addresses for Nominatim API query
# create a new column 'Full Address'
df_1['Full Address'] = df_1['Street'] + ', ' + df_1['House Number'].astype(str) + ', ' + df_1['City'] + ', ' + df_1['Postcode'].astype(str) + ', Germany'

In [9]:
# initialise geocoder Nominatim
geolocator = Nominatim(user_agent="your_app_name")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [10]:
# a function takes a row
# applies the geolocator.geocode function from the Nominatim API to each row
# gets the geographic coordinates to each row
def my_geocoder(row):
    try:
        point = geolocator.geocode(row).point
        return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})
    except:
        return None

# applies 'my_geocoder' function to each row in the df_1 dataframe
# extracts the value from the 'Full Address' column and passes it to 'my_geocoder' function
# finds the corresponding latitude and longitude of the full address
# adds the results to the new columns 'Latitude' and 'Longitude'
df_1[['Latitude', 'Longitude']] = df_1.apply(lambda x: my_geocoder(x['Full Address']), axis=1)

In [11]:
# 'zip' function pairs elements together and creates a tuple (longitude, latitude)
# 'Point(xy)' creates a Point object for each tuple, using Shapely library
# creates a list of points for each row in df_1
geometry = [Point(xy) for xy in zip(df_1['Longitude'], df_1['Latitude'])]

In [12]:
# converts 'df_1' dataframe into a 'gdf_1' GeoDataFrame
gdf_1 = gpd.GeoDataFrame(df_1, geometry=geometry)

# sets crs to epsg=4326
gdf_1.set_crs(epsg=4326, inplace=True)

,MaStR-Nr. der Stromerzeugungseinheit,Anlagenbetreiber,Anzeige-Name der Stromerzeugungseinheit,Postcode,City,Street,House Number,State,Datum der erstmaligen Inbetriebnahme der Einheit,Jahr der Inbetriebnahme der Einheit,...,Ist die Stromerzeugungseinheit ein Bestandteil eines Grenzkraftwerkes?: ja \nNettonennleistung der Einspeisung in ein deutsches Netz:,Technologie der Stromerzeugung,Volleinspeisung oder Teileinspeisung?,Anschlussnetzbetreiber,Spannungsebene,Source,Full Address,Latitude,Longitude,geometry
17,SEE909373287806,Heizkraftwerk Altenstadt GmbH & Co. KG,Heizkraftwerk Altenstadt,86972,Altenstadt,Triebstraße,90,Bayern,1999-10-01,1999.0,...,NaN,Kondensationsmaschine mit Entnahme,Teileinspeisung (einschließlich Eigenverbrauch),LEW Verteilnetz GmbH (SNB911705062982),Mittelspannung,bundesnetzagentur_power_plants.xlsx,"Triebstraße, 90, Altenstadt, 86972, Germany",47.797296,10.859082,POINT (10.85908 47.79730)
18,SEE950208762586,Untere Iller GmbH,Untereichen F1,89281,Altenstadt,Werkstraße,60,Bayern,1931-01-01,1931.0,...,NaN,NaN,Teileinspeisung (einschließlich Eigenverbrauch),LEW Verteilnetz GmbH (SNB911705062982),Umspannebene Mittelspannung/Niederspannung,bundesnetzagentur_power_plants.xlsx,"Werkstraße, 60, Altenstadt, 89281, Germany",NaN,NaN,POINT EMPTY
19,SEE968240296907,Untere Iller GmbH,Untereichen F2,89281,Altenstadt,Werkstraße,60,Bayern,1931-01-01,1931.0,...,NaN,NaN,Teileinspeisung (einschließlich Eigenverbrauch),LEW Verteilnetz GmbH (SNB911705062982),Umspannebene Mittelspannung/Niederspannung,bundesnetzagentur_power_plants.xlsx,"Werkstraße, 60, Altenstadt, 89281, Germany",NaN,NaN,POINT EMPTY
20,SEE968986286460,Untere Iller GmbH,Untereichen F3,89281,Altenstadt,Werkstraße,60,Bayern,1931-01-01,1931.0,...,NaN,NaN,Teileinspeisung (einschließlich Eigenverbrauch),LEW Verteilnetz GmbH (SNB911705062982),Umspannebene Mittelspannung/Niederspannung,bundesnetzagentur_power_plants.xlsx,"Werkstraße, 60, Altenstadt, 89281, Germany",48.170123,10.104592,POINT (10.10459 48.17012)
33,SEE973646665912,DS Smith Paper Deutschland GmbH,BHKW 1,63741,Aschaffenburg,Weichertstraße,7,Bayern,2005-12-14,2005.0,...,NaN,Verbrennungsmotor,Volleinspeisung,Aschaffenburger Versorgungs-GmbH (SNB911081401...,Umspannebene Hochspannung/Mittelspannung; Umsp...,bundesnetzagentur_power_plants.xlsx,"Weichertstraße, 7, Aschaffenburg, 63741, Germany",NaN,NaN,POINT EMPTY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1927,SEE9-Dummy-EEG,EEG-Anlagen < 10 MW,NaN,NaN,NaN,NaN,NaN,Bayern,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,bundesnetzagentur_power_plants.xlsx,NaN,46.314475,11.048029,POINT (11.04803 46.31448)
1943,SEE9-Dummy-EEG,EEG-Anlagen < 10 MW,NaN,NaN,NaN,NaN,NaN,Bayern,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,bundesnetzagentur_power_plants.xlsx,NaN,46.314475,11.048029,POINT (11.04803 46.31448)
1962,SEE9-Dummy-EEG,EEG-Anlagen < 10 MW,NaN,NaN,NaN,NaN,NaN,Bayern,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,bundesnetzagentur_power_plants.xlsx,NaN,46.314475,11.048029,POINT (11.04803 46.31448)
1978,SEE9-Dummy-EEG,EEG-Anlagen < 10 MW,NaN,NaN,NaN,NaN,NaN,Bayern,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,bundesnetzagentur_power_plants.xlsx,NaN,46.314475,11.048029,POINT (11.04803 46.31448)


In [13]:
# applies the 'clear_column_names' function created in 'utiles.py':
# replaces umlauts with their ASCII equivalents,
# removes parenthesis, colons, soft hyphens and other special characters
clear_column_names(gdf_1)

In [14]:
# before saving the resulting geodataframe in a shapefile,
# datetime objects YYYY-MM-DD should be converted into string objects 'YYYY-MM-DD',
# because a shapefile format doesn't support a 'datetime' format
gdf_1['Datum der erstmaligen Inbetriebnahme der Einheit'] = gdf_1['Datum der erstmaligen Inbetriebnahme der Einheit'].dt.strftime('%Y-%m-%d')

In [15]:
# saves gdf_1 to a shapefile 'bundesnetzagentur_power_plants.shp'
gdf_1.to_file('data/sources/bundesnetzagentur_power_plants.shp', driver='ESRI Shapefile')

C:\Users\Natalia\AppData\Local\Temp\ipykernel_21096\1546126883.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_1.to_file('data/sources/bundesnetzagentur_power_plants.shp', driver='ESRI Shapefile')


2 Open Power System Data (OPSD)




In [16]:
# read an Excel file 'OPSD_conventional_power_plants.xlsx'
df_2 = pd.read_excel('data/sources/Open Power System Data_OPSD/OPSD_conventional_power_plants.xlsx')

In [17]:
# Extract the file name
file_name_2 = os.path.basename('data/sources/Open Power System Data_OPSD/OPSD_conventional_power_plants.xlsx')
# Add a new column with the file name
df_2['Source'] = file_name_2

In [18]:
# keep data only for Bavaria
df_2 = df_2[df_2['state'] == 'Bayern']

In [19]:
# 'zip' function pairs elements together and creates a tuple (longitude, latitude)
# 'Point(xy)' creates a Point object for each tuple, using Shapely library
# creates a list of points for each row in df_2
geometry = [Point(xy) for xy in zip(df_2['lon'], df_2['lat'])]

In [20]:
# convert the dataframe df_2 to a geodataframe gdf_2 by adding 'geometry' column
gdf_2 = gpd.GeoDataFrame(df_2, geometry=geometry)

In [21]:
# set crs epsg=4326
gdf_2.set_crs(epsg=4326, inplace=True)

,id,name_bnetza,block_bnetza,name_uba,company,street,postcode,city,state,country,...,energy_source_level_2,energy_source_level_3,eeg,network_node,voltage,network_operator,merge_comment,comment,Source,geometry
1,BNA0021,HKW Altenstadt,NaN,NaN,Heizkraftwerk Altenstadt GmbH & Co. KG,Triebstraße 90,86972,Altenstadt,Bayern,DE,...,Bioenergy,Biomass and biogas,yes,NaN,Mittelspannung (MS),LEW Verteilnetz GmbH,NaN,NaN,OPSD_conventional_power_plants.xlsx,POINT (10.85874 47.79714)
23,BNA0553,BMHKW-BinderHolz,NaN,NaN,BinderHolz Deutschland GmbH,Einsteinstraße 9,85092,Kösching,Bayern,DE,...,Bioenergy,Biomass and biogas,yes,UW GROM,Hochspannung (HS),Bayernwerk AG,NaN,NaN,OPSD_conventional_power_plants.xlsx,POINT (11.50679 48.79123)
30,BNA0692,KWK Neumarkt,NaN,NaN,Pfleiderer Neumarkt GmbH,Dreichlinger Str. 76,92318,Neumarkt i.d. Oberpfalz,Bayern,DE,...,Bioenergy,Biomass and biogas,yes,Übergabe KWK,Mittelspannung (MS),Stadtwerke Neumarkt i.d.OPf.,NaN,NaN,OPSD_conventional_power_plants.xlsx,POINT (11.45160 49.26133)
34,BNA0926a,Heizkraftwerk der Sappi Stockstadt GmbH,Biomasseheizkraftwerk,NaN,Sappi Stockstadt GmbH,Obernburger Straße 1-9,63811,Stockstadt,Bayern,DE,...,Bioenergy,Biomass and biogas,yes,Primär Kundenanlage zur betrieblichen Eigenver...,Hochspannung (HS),Westnetz GmbH,NaN,NaN,OPSD_conventional_power_plants.xlsx,POINT (9.06872 49.97745)
36,BNA1091,Zolling,BMHKW,NaN,ENGIE Deutschland AG,Leininger Str. 1,85406,Zolling,Bayern,DE,...,Bioenergy,Biomass and biogas,yes,110-kV-Schaltfeld E05 / LS 173,Hochspannung (HS),Bayernwerk AG,NaN,NaN,OPSD_conventional_power_plants.xlsx,POINT (11.80190 48.45534)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
870,BNA1161,MVA Ingolstadt,Müllheizkraftwerk (MHKW),NaN,Zweckverband Müllverwertungsanlage Ingolstadt ...,Am Mailinger Bach 141,85055,Ingolstadt,Bayern,DE,...,Waste,NaN,no,DE000314850550000102762000000S013,Mittelspannung (MS),Stadtwerke Ingolstadt Netze GmbH,NaN,NaN,OPSD_conventional_power_plants.xlsx,POINT (11.49185 48.76450)
889,BNA1254,Müllkraftwerk Schwandorf,NaN,NaN,Zweckverband Müllverwertung Schwandorf,Alustraße 7,92421,Schwandorf,Bayern,DE,...,Waste,NaN,no,UW Schwandorf,Hochspannung (HS),Bayernwerk AG,NaN,NaN,OPSD_conventional_power_plants.xlsx,POINT (12.08795 49.30998)
895,BNA1295,AVA GmbH,AHKW,NaN,AVA Abfallverwertung Augsburg GmbH,Am Mittleren Moos 60,86167,Augsburg,Bayern,DE,...,Waste,NaN,no,V0 BBC06,Hochspannung (HS),Netze Augsburg GmbH,NaN,NaN,OPSD_conventional_power_plants.xlsx,POINT (10.93560 48.40113)
899,BNA1449a,Turbosatz 1,NaN,NaN,Zweckverband Abfallwirtschaft Raum Würzburg,Gattingerstr. 31,97076,Würzburg,Bayern,DE,...,Waste,NaN,no,SH 399 MHKW,Mittelspannung (MS),Mainfranken Netze GmbH,NaN,NaN,OPSD_conventional_power_plants.xlsx,POINT (9.99406 49.79300)


In [22]:
# saves the result to a shapefile 'OPSD_conventional_power_plants.shp'
gdf_2.to_file('data/sources/OPSD_conventional_power_plants.shp', driver='ESRI Shapefile')

C:\Users\Natalia\AppData\Local\Temp\ipykernel_21096\447393407.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_2.to_file('data/sources/OPSD_conventional_power_plants.shp', driver='ESRI Shapefile')


3 Energieatlas




In [23]:
# 7 Excel files from the Energieatlas Bayern of the power plants of different types:
# biomass, biomethane, fossil, geothermal, kwk, solarthermal, waste.

In [24]:
# Each file has a column with geometry data 'Geometrie (EWKT)' in a string format
# containing crs and a point object: 'SRID=31468;POINT (4575128.048489725 5347063.698786772)'.
# SRID=31468 refers to the Gauss-Krüger projection for the Bavarian region in Germany,
# which uses meters as the unit of measurement.

# The function 'extract_geometry' takes a string,
# extract the geometry coordinates from it using a function 'loads' from shapely.wkt module,
# recognizes a well-known text (wkt) and takes geometry from it.
def extract_geometry(geom_str):
    # splits a string by semicolon, takes the 2nd element 'POINT (4575128.048489725 5347063.698786772)' which is a string
    wkt_str = geom_str.split(';')[1]
    # uses 'loads' function to convert the wkt string into a geometry object
    return loads(wkt_str)

In [25]:
# Creates a list of 7 Excel files
files = [
    'data/sources/Energieatlas/energieatlas_biomass.xlsx',
    'data/sources/Energieatlas/energieatlas_biomethane.xlsx',
    'data/sources/Energieatlas/energieatlas_fossil.xlsx',
    'data/sources/Energieatlas/energieatlas_geothermal.xlsx',
    'data/sources/Energieatlas/energieatlas_kwk.xlsx',
    'data/sources/Energieatlas/energieatlas_solarthermal.xlsx',
    'data/sources/Energieatlas/energieatlas_waste.xlsx'
]

In [26]:
# Processes multiple Excel files in a loop:
# reads, extracts geometry, creates a gpd, sets crs, clears column names, saves in .shp
for i, file in enumerate(files):
    df_3 = pd.read_excel(file)
    file_name_3 = os.path.basename(file)
    df_3['Source'] = file_name_3
    df_3['geometry'] = df_3['Geometrie (EWKT)'].apply(extract_geometry)
    gdf_3 = gpd.GeoDataFrame(df_3, geometry='geometry')
    gdf_3.set_crs(epsg=31468, inplace=True)
    gdf_3 = gdf_3.to_crs(epsg=4326)

    # applies the 'clear_column_names' function created in 'utiles.py':
    # replaces umlauts with their ASCII equivalents,
    # removes parenthesis, colons, soft hyphens and other special characters
    clear_column_names(gdf_3)

    file_name = os.path.splitext(os.path.basename(file))[0]
    output_file = f'data/sources/{file_name}.shp'
    gdf_3.to_file(output_file, driver='ESRI Shapefile')

C:\Users\Natalia\anaconda3\envs\geoanalysis\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\Natalia\AppData\Local\Temp\ipykernel_21096\3533636678.py:19: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_3.to_file(output_file, driver='ESRI Shapefile')
C:\Users\Natalia\anaconda3\envs\geoanalysis\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\Natalia\AppData\Local\Temp\ipykernel_21096\3533636678.py:19: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_3.to_file(output_file, driver='ESRI Shapefile')
C:\Users\Natalia\anaconda3\envs\geoanalysis\Lib\site-packages\openpy